## Dynamic Expressions

### Define wc_lang math expressions

In [1]:
import warnings
warnings.simplefilter("ignore")
import wc_lang
from wc_lang import (Model, SpeciesType, Species, Compartment, Parameter, Function, FunctionExpression, Observable,
                     StopCondition, ExpressionMethods)

def make_test_reference_objects():
    model = Model()
    comp = model.compartments.create(id='comp')
    objects = {
        Observable: {},
        Parameter: {},
        Function: {},
        Species: {},
    }
    for id in ['a', 'b', 'duped_id']:
        objects[Parameter][id] = model.parameters.create(id=id)
    for id in ['ccc', 'ddd', 'duped_id']:
        objects[Observable][id] = model.observables.create(id=id)
    for id in ['f', 'g', 'duped_id']:
        objects[Function][id] = model.functions.create(id=id)
    species_types = []
    for i in range(4):
        species_type = model.species_types.create(id='spec_type_{}'.format(i))
        species = Species(species_type=species_type, compartment=comp)
        objects[Species][species.get_id()] = species
    return model, objects

model, objects = make_test_reference_objects()

### Function

In [2]:
# Function can reference Observables, Parameters, and other Functions, and it can use some Python functions
fun_1 = ExpressionMethods.make_obj(model, Function, 'fun_1', 'ccc + max(a, ddd)', objects)
# make_obj has this signature:
# def make_obj(model, model_type, id, expression, objects)
print('id:', fun_1.id)

id: fun_1


In [3]:
# each math expression has a corresponding Expression object
print('type of expression:', type(fun_1.expression).__name__)
# math expressions that can use Python functions have a list of valid functions in Meta
print('valid_functions:', FunctionExpression.Meta.valid_functions)

type of expression: FunctionExpression
valid_functions: (<built-in function ceil>, <built-in function floor>, <built-in function exp>, <built-in function pow>, <built-in function log>, <built-in function log10>, <built-in function min>, <built-in function max>)


In [4]:
# references to the objects used are stored in a related attribute
print('expression:', fun_1.expression.expression)
for attr in ['observables', 'parameters', 'functions']:
    print("{}: {}".format(attr, [obj.get_id() for obj in getattr(fun_1.expression, attr)]))

expression: ccc + max(a, ddd)
observables: ['ccc', 'ddd']
parameters: ['a']
functions: []


In [5]:
# FunctionExpression contains an 'analyzed_expr', which is a tokenized, validated expression
# it contains the Python tokens from the expression ...
print('analyzed_expr tokens:', [t.string for t in fun_1.expression.analyzed_expr.tokens])

# wc_tokens, which contain the interpretation of each Python token ...
print('\nanalyzed_expr wc_tokens:\n',
      '\n'.join(['  '+str(wc_token) for wc_token in fun_1.expression.analyzed_expr.wc_tokens]))

analyzed_expr tokens: ['ccc', '+', 'max', '(', 'a', ',', 'ddd', ')']

analyzed_expr wc_tokens:
   WcLangToken(tok_code=<TokCodes.wc_lang_obj_id: 1>, token_string='ccc', model_type=<class 'wc_lang.core.Observable'>, model_id='ccc', model=<wc_lang.core.Observable object at 0x7fc04c0cc748>)
  WcLangToken(tok_code=<TokCodes.op: 4>, token_string='+', model_type=None, model_id=None, model=None)
  WcLangToken(tok_code=<TokCodes.math_fun_id: 2>, token_string='max', model_type=None, model_id=None, model=None)
  WcLangToken(tok_code=<TokCodes.op: 4>, token_string='(', model_type=None, model_id=None, model=None)
  WcLangToken(tok_code=<TokCodes.wc_lang_obj_id: 1>, token_string='a', model_type=<class 'wc_lang.core.Parameter'>, model_id='a', model=<wc_lang.core.Parameter object at 0x7fc04c100828>)
  WcLangToken(tok_code=<TokCodes.op: 4>, token_string=',', model_type=None, model_id=None, model=None)
  WcLangToken(tok_code=<TokCodes.wc_lang_obj_id: 1>, token_string='ddd', model_type=<class 'wc_lang.c

### Stop condition

In [6]:
# StopCondition is like Function, but it must return a boolean
sc_1 = ExpressionMethods.make_obj(model, StopCondition, 'sc 1', '1 < a + ddd', objects)
print(sc_1)

<wc_lang.core.StopCondition: sc 1>


In [7]:
# expressions are validated by executing a test Python evaluation
# if a StopCondition doesn't return a boolean its validation fails
print(ExpressionMethods.make_obj(model, StopCondition, 'sc 1', 'a + ddd', objects))
# the test evaluation assumes that all referenced objects have values of 1

'expression':
  Evaluating 'a + ddd', a StopConditionExpression expression, should return a bool but it returns a float


### Observable

In [8]:
# Observables can reference Species and other Observables
ccc = ExpressionMethods.make_obj(model, Observable, 'ccc', 'ccc + ddd - 2 * spec_type_0[comp]', objects)
print(ccc)

# cycles aren't allowed, but they cannot be detected until all expression have been made

<wc_lang.core.Observable: ccc>
